# Deep Neural Nertwork

In this notebook we will explore we will explore Deep Neural Networks (DNN) and its parameters using Keras library.

## Brief History

To better understand DNN let's first go over the beginings of neural networks.

### What is a Neuron?

In 1943 Warren McCulloch and Walter Pitts published the first concept of a neuron. Neurons are interconnected nerve cells in the brain that process chemical and electrical signals. [Source](https://learning.oreilly.com/library/view/python-machine-learning/9781787125933/ch02s02.html)

In [1]:
%%html
<img src='brain_cell.png', width=500, height=500>

In 1957 Frank Rosenblatt published the `perceptron learning rule` based on the neuron concept. Such rule gave rise to **perceptron**, a binary classification algorithm that automatically learns the optimal coefficients for each input called to make a decision. In 1960, Widrow and Hoff published the **ADAtive LInear NEuron (Adaline)**, a modification of the perceptron with an activation function in between the net input function and the threshold function. In contrast to the perceptron algorithm, the cost function of adaline is differentiable. This property allows to use an optimization algorithm termed **gradient descent** to find the best weights and minimize the error.

In [2]:
%%html
<img src='Neuron.png', width=500, height=500>

In [3]:
%%html
<img src='gradientDescent.png', width=500, height=500>

### Multi-layer Neural Network

In 1986 the **back propagation** algorithm was published which proved to train neural networks more efficiently. Back propagation takes the gradients with respect to each wegiht for each node in each layer and updates the weights simultanoiuslly based on the error.

In [4]:
%%html
<img src='Multi_NN.png', width=700, height=700>

## DNN Classification of Breast Cancer

In this section we will build a DNN to classify if the type of breast cancer is malignant or benign. Additional information is found below the print function of the description (`DESCR`) of the data.

In [78]:
# libraries
import os, shutil

from sklearn import datasets
from matplotlib import pyplot as plt

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense

In [6]:
cancer = datasets.load_breast_cancer()

In [7]:
cancer.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [8]:
cancer['target_names']

array(['malignant', 'benign'], dtype='<U9')

In [9]:
cancer['feature_names']

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

In [12]:
print(cancer['DESCR'])

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry 
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 3 is Mean Radius, f

In [13]:
cancer_df = pd.DataFrame(cancer['data'], columns=cancer['feature_names'])

In [14]:
cancer_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [15]:
cancer_df.shape

(569, 30)

In [16]:
cancer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 30 columns):
mean radius                569 non-null float64
mean texture               569 non-null float64
mean perimeter             569 non-null float64
mean area                  569 non-null float64
mean smoothness            569 non-null float64
mean compactness           569 non-null float64
mean concavity             569 non-null float64
mean concave points        569 non-null float64
mean symmetry              569 non-null float64
mean fractal dimension     569 non-null float64
radius error               569 non-null float64
texture error              569 non-null float64
perimeter error            569 non-null float64
area error                 569 non-null float64
smoothness error           569 non-null float64
compactness error          569 non-null float64
concavity error            569 non-null float64
concave points error       569 non-null float64
symmetry error             569 

In [17]:
target = cancer['target']

In [18]:
target.shape

(569,)

In [24]:
# target

In [21]:
x_train, x_test, y_train, y_test = train_test_split(cancer_df, target, stratify=target, random_state=42, train_size=0.75)

In [22]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((426, 30), (143, 30), (426,), (143,))

In [47]:
scaler = StandardScaler()

In [48]:
x_train, x_test = scaler.fit_transform(x_train), scaler.transform(x_test)

In [49]:
# make DNN model

model = Sequential()
model.add(Dense(50, activation='relu', input_dim=30))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['acc'])

In [50]:
model.fit(x_train, y_train, epochs=100, validation_split=0.2, verbose=0)

In [51]:
model.evaluate(x_train, y_train)

426/426 [==============================] - 0s 18us/step


[0.08058169882901957, 0.9812206625938416]

In [52]:
model.evaluate(x_test, y_test)

143/143 [==============================] - 0s 26us/step


[0.09723454609617486, 0.9790209531784058]

### Image Classification

In [53]:
orig_data = '/Users/carlostavarez/Desktop/dogs-vs-cats/train'

In [56]:
len(os.listdir(orig_data))

25000

In [62]:
dogs_cats

'/Users/carlostavarez/Documents/GitHub/2020-spring-datascience/Week6_4.23'

In [97]:
dire = '/Users/carlostavarez/Documents/GitHub/2020-spring-datascience/Week6_4.23'

In [98]:
# set dirs

base_dir = os.path.join(dire, 'dogs_cats')
train_dir = os.path.join(base_dir, 'train')
valid_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')

In [99]:
os.mkdir(base_dir)
os.mkdir(train_dir)
os.mkdir(valid_dir)
os.mkdir(test_dir)

In [100]:
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

valid_cats_dir = os.path.join(valid_dir, 'cats')
valid_dogs_dir = os.path.join(valid_dir, 'dogs')

test_cats_dir = os.path.join(test_dir, 'cats')
test_dogs_dir = os.path.join(test_dir, 'dogs')

In [101]:
os.mkdir(train_cats_dir)
os.mkdir(train_dogs_dir)

os.mkdir(valid_cats_dir)
os.mkdir(valid_dogs_dir)

os.mkdir(test_cats_dir)
os.mkdir(test_dogs_dir)

In [102]:
def move_file(send_to_dir, fname_list):
    
    for fname in fname_list:
        
        src = os.path.join(orig_data, fname)
        dst = os.path.join(send_to_dir, fname)
        shutil.copyfile(src, dst)

In [103]:
def make_animal(animal, sub_sample_tr, sub_sample):
    
    train_list = [animal + '.{}.jpg'.format(i) for i in range(1000)]
    valid_list = [animal + '.{}.jpg'.format(i) for i in range(1000, 1500)]
    test_list = [animal + '.{}.jpg'.format(i) for i in range(1500, 2000)]
    
    train_list = [train_list[idx] for idx in np.random.choice(range(1000), sub_sample_tr, replace=False)]
    valid_list = [valid_list[idx] for idx in np.random.choice(range(500), sub_sample, replace=False)]
    test_list = [test_list[idx] for idx in np.random.choice(range(500), sub_sample, replace=False)]
    
    return [train_list, valid_list, test_list]

In [108]:
def move_animal(animal, list_list, sub_sample_tr, sub_sample):
    
    animal_list = make_animal(animal, sub_sample_tr, sub_sample)
    
    for ani, ani_list in zip(animal_list, list_list):
        
        move_file(ani_list, ani)

In [109]:
cats_list = [train_cats_dir, valid_cats_dir, test_cats_dir]

In [110]:
dogs_list = [train_dogs_dir, valid_dogs_dir, test_dogs_dir]

In [111]:
move_animal('cat', cats_list, 500, 100)

In [112]:
move_animal('dog', dogs_list, 500, 100)